# Lego ID Scraper

To get the IDs from the Brickset website I scrape them from the "list by year" on the website
It's trivial to set the year below to a specific year, or to set it to current year to grab new sets for the data.

## First library imports and setting up Selenium options


In [1]:
import os
import numpy as np
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from tqdm.notebook import tqdm

print('Packages Imported')

Packages Imported


Setting up the Selenium options here, setting it to headless and using the Chrome webdriver

In [63]:
#selenium setup
def selenium_setup():
    options = Options()

    options.add_argument("--window-size=1920,1200")
    options.add_argument('--headless')
    return webdriver.Chrome(options=options)

## Main loop
Itterates through the year list and pagelist to get the IDs.

2022 has the most pages with 39. I could shorten this down so that each year only uses a certain number of pages associated with it, but this will do for now, as querying empty pages does not take much time.

In [20]:

driver = selenium_setup()
years = range(1949,2024)

ID_list = []
pages = range(1,41)
outter_bar = tqdm(years, desc = 'Years',mininterval=0,miniters=1)
inner_bar = tqdm(pages, desc = 'Pages',mininterval=0,miniters=1, leave = True)

for year in years:    
    outter_bar.update(1)
    for page in pages:

        inner_bar.update(1)
        driver.get('https://brickset.com/sets/year-' +str(year)+'/page-' +str(page))
        pageSource = driver.page_source

        ID = driver.find_elements(By.XPATH, "//div[contains(@class, 'tags')]/a[1][contains(.,'-')]")
        for i in range(0,len(ID)):
            if ID[i].text !='':
                ID_list.append(ID[i].text)
    inner_bar.reset()
print("Complete!")


Years:   0%|          | 0/75 [00:00<?, ?it/s]

Pages:   0%|          | 0/40 [00:00<?, ?it/s]

Complete!


## Print to CSV file

In [10]:

df = pd.DataFrame(ID_list)
df.to_csv('SetIDs.csv', sep = ',', index=False,header=False)


## Update 


In [104]:
years = [2023]
driver = selenium_setup()
if os.path.exists('SetIDs.csv'):
    IDs_df = pd.read_csv('SetIDs.csv', index_col=False,header=None)
    new_ID_list = []
    pages = range(1,41)
    outter_bar = tqdm(years, desc = 'Years',mininterval=0,miniters=1)
    inner_bar = tqdm(pages, desc = 'Pages',mininterval=0,miniters=1, leave = True)

    for year in years:    
        outter_bar.update(1)
        for page in pages:

            inner_bar.update(1)
            driver.get('https://brickset.com/sets/year-' +str(year)+'/page-' +str(page))
            pageSource = driver.page_source

            ID = driver.find_elements(By.XPATH, "//div[contains(@class, 'tags')]/a[1][contains(.,'-')]")
            for i in range(0,len(ID)):
                if ID[i].text !='' and ~IDs_df[0].str.contains(ID[i].text).any():
                    
                    new_ID_list.append(ID[i].text)

        inner_bar.reset()
    if len(new_ID_list) >0:
        new_ID_df =   pd.DataFrame(new_ID_list) 
        FullIDs  = pd.concat([IDs_df,new_ID_df])    
        FullIDs.to_csv('SetIDs.csv', sep = ',', index=False,header=False)
    else:
        print("No new IDs")
    print("Complete!")
    
else:
   print("Perform initial run!")

Years:   0%|          | 0/1 [00:00<?, ?it/s]

Pages:   0%|          | 0/40 [00:00<?, ?it/s]

Complete!
